In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import ipywidgets as widgets
import matplotlib.animation as animation
import matplotlib.gridspec as gridspec
from pathlib import Path

from IPython.display import display
from HMM_target_detector_20251204_UPDATED_with_target_change import HMM_target_detector

In [57]:
ANGLE_RANGE = 2*np.pi
CODE_COLOR_MAP = {0 : 'yellow',
                  1 : 'blue',
                  2 : 'purple'}

In [61]:
prob_code0_in_states = np.array([1, 0, 1, 1])
prob_code1_in_states = np.array([0, 1, 0, 0])
EMISSION_MATRIX_TARGET1 = np.vstack([prob_code0_in_states, prob_code1_in_states])

prob_code0_in_states = np.array([1, 1, 1, 1])
prob_code1_in_states = np.array([0, 0, 0, 0])
EMISSION_MATRIX_TARGET2 = np.vstack([prob_code0_in_states, prob_code1_in_states])

prob_code0_in_states = np.array([1, 0, 1, 0])
prob_code1_in_states = np.array([0, 1, 0, 1])
EMISSION_MATRIX_TARGET3 = np.vstack([prob_code0_in_states, prob_code1_in_states])

EMISSION_MATRIX_TARGETS = np.stack([EMISSION_MATRIX_TARGET1, EMISSION_MATRIX_TARGET2, EMISSION_MATRIX_TARGET3])
EMISSION_MATRIX_TARGETS, EMISSION_MATRIX_TARGETS.shape

(array([[[1, 0, 1, 1],
         [0, 1, 0, 0]],
 
        [[1, 1, 1, 1],
         [0, 0, 0, 0]],
 
        [[1, 0, 1, 0],
         [0, 1, 0, 1]]]),
 (3, 2, 4))

In [62]:
agent = HMM_target_detector(emission_matrix_targets=EMISSION_MATRIX_TARGETS)

In [63]:
initial_angle_slider = widgets.IntSlider(
    value=0,  
    min=0,
    max=360,
    description="Initial Angle",
    style={'description_width': 'initial'},
    continuous_update=True,
    layout=widgets.Layout(width="600px"),
)

interactive_plot = widgets.interactive(
    agent.observe_angle_state_pair, initial_angle=initial_angle_slider)
display(interactive_plot)

interactive(children=(IntSlider(value=0, description='Initial Angle', layout=Layout(width='600px'), max=360, s…

In [64]:
save_folder = Path(f'../test_trials__detection_model_20251204_223100_four_states')
save_folder.mkdir(parents=True, exist_ok=True)

In [67]:
def get_deterministic_target_transition_matrix(forecast_t_candidate_step):
    if forecast_t_candidate_step == 'switch':
        target_transition_matrix = np.array([[0, 1, 0], 
                                             [0, 0, 1],
                                             [1, 0, 0]])
    else:
        target_transition_matrix = np.array([[1, 0, 0], 
                                             [0, 1, 0],
                                             [0, 0, 1]])
    return target_transition_matrix

In [99]:
def compute_forecasted_entropy_from_forecasted_obs(agent, state_transition_matrix, target_transition_matrix):
    updated_posteriors_across_targets_and_obs = np.zeros((agent.num_observations, agent.num_beliefs))
    normalized_updated_posteriors_across_targets_and_obs = np.zeros((agent.num_observations, agent.num_beliefs))
    prob_obs_at_forecast_t_given_obs_at_cur_t = np.zeros(agent.num_observations)
    updated_posteriors_across_targets_and_obs, normalized_updated_posteriors_across_targets_and_obs

    for target_t in range(agent.num_beliefs):
        subterm1 = agent.within_trial_arrays['joint_prob_received_obs_state_t_from_targets'][agent.within_trial_params["time_ind"]] @ state_transition_matrix
        subterm2 = (agent.emission_matrix_targets[target_t,:] @ subterm1.T)
        likelihood_obs_from_target_forecast_t_fromtargets = subterm2 @ target_transition_matrix[:,target_t]
        term2 = agent.within_trial_arrays['likelihood_received_obs_from_targets_t'][agent.within_trial_params["time_ind"], target_t]
        likelihood_obs_forecast_t_given_received_obs = agent.update_likelihood_given_observed_sequence(likelihood_obs_from_target_forecast_t_fromtargets, term2)

        updated_belief_after_action = agent.within_trial_arrays['posterior_t'][agent.within_trial_params["time_ind"]] @ target_transition_matrix
        updated_posteriors_across_targets_and_obs[:,target_t] = updated_belief_after_action[target_t] * likelihood_obs_from_target_forecast_t_fromtargets

    prob_obs_at_forecast_t_given_obs_at_cur_t = updated_posteriors_across_targets_and_obs.sum(axis=1)
    forecasted_entropy_across_obs = np.zeros(agent.num_observations)
    for target_t in range(agent.num_beliefs):
        normalized_updated_posterior = updated_posteriors_across_targets_and_obs[:,target_t] / prob_obs_at_forecast_t_given_obs_at_cur_t
        normalized_updated_posteriors_across_targets_and_obs[:,target_t] = normalized_updated_posterior

        for obs in range(agent.num_observations):
            if normalized_updated_posterior[obs] > 0:
                forecasted_entropy_across_obs[obs] += (-1*normalized_updated_posterior[obs]*np.log2(normalized_updated_posterior[obs]))

    expected_entropy_across_obs = prob_obs_at_forecast_t_given_obs_at_cur_t @ forecasted_entropy_across_obs.reshape(agent.num_observations, 1)
    return prob_obs_at_forecast_t_given_obs_at_cur_t, expected_entropy_across_obs

In [100]:
agent = HMM_target_detector(emission_matrix_targets=EMISSION_MATRIX_TARGETS)
agent.move_and_sample_from_object()

agent.update_posterior_and_compute_current_entropy()

agent.debug_within_trial_print_statements()

Current angle: 0.0
Code received: 0.0
[0.25       0.33333333 0.16666667]
P(O1=[0.],S1|T1)=[0.25 0.   0.25 0.25], P(O1=[0.]|T1)=0.75, P(S1|O1=[0.],T1)=[0.33 0.   0.33 0.33], P(T|O1=[0.])=[0.33333333 0.44444444 0.22222222], H1=1.53
P(O1=[0.],S1|T2)=[0.25 0.25 0.25 0.25], P(O1=[0.]|T2)=1.0, P(S1|O1=[0.],T2)=[0.25 0.25 0.25 0.25], P(T|O1=[0.])=[0.33333333 0.44444444 0.22222222], H1=1.53
P(O1=[0.],S1|T3)=[0.25 0.   0.25 0.  ], P(O1=[0.]|T3)=0.5, P(S1|O1=[0.],T3)=[0.5 0.  0.5 0. ], P(T|O1=[0.])=[0.33333333 0.44444444 0.22222222], H1=1.53


In [101]:
agent.within_trial_arrays['posterior_t'][agent.within_trial_params["time_ind"]]

array([0.33333333, 0.44444444, 0.22222222])

In [107]:
for i, forecast_t_candidate_step in enumerate(agent.candidate_steps):
    forecast_t_candidate_step_states = (forecast_t_candidate_step[0]).astype(float)
    forecast_t_candidate_step_targets = str(forecast_t_candidate_step[1])
    state_transition_matrix = (agent.get_deterministic_state_transition_matrix(forecast_t_candidate_step_states))
    target_transition_matrix = (get_deterministic_target_transition_matrix(forecast_t_candidate_step_targets))
    prob_obs_at_forecast_t_given_obs_at_cur_t, expected_forecast_t_S = compute_forecasted_entropy_from_forecasted_obs(agent, state_transition_matrix, target_transition_matrix)
    agent.within_trial_arrays['expected_forecast_t_S_per_step'][agent.within_trial_params["time_ind"], i] = expected_forecast_t_S
    
state_transition_matrix, target_transition_matrix

/var/folders/qj/4txkrxcn2ld7mby9_6n6npnr0000gn/T/ipykernel_89692/2181716830.py:7: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  agent.within_trial_arrays['expected_forecast_t_S_per_step'][agent.within_trial_params["time_ind"], i] = expected_forecast_t_S
/var/folders/qj/4txkrxcn2ld7mby9_6n6npnr0000gn/T/ipykernel_89692/925695447.py:20: RuntimeWarning: invalid value encountered in divide
  normalized_updated_posterior = updated_posteriors_across_targets_and_obs[:,target_t] / prob_obs_at_forecast_t_given_obs_at_cur_t


(array([[0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [1., 0., 0., 0.]]),
 array([[1, 0, 0],
        [0, 1, 0],
        [0, 0, 1]]))

In [109]:
agent.within_trial_arrays['expected_forecast_t_S_per_step'][agent.within_trial_params["time_ind"]]

array([0.92779385, 0.88081502, 0.92779385, 0.70817549, 1.12089233,
       0.70817549])

In [ ]:
# agent.print_debug_forecasted_entropy_statements(prob_obs_at_forecast_t_given_obs_at_cur_t, expected_forecast_t_S)

In [ ]:
def compute_forecasted_entropy_from_forecasted_obs(agent, state_transition_matrix, target_transition_matrix):
    forecasted_S = np.zeros(agent.num_observations)
    prob_obs_at_forecast_t_given_obs_at_cur_t = np.zeros(agent.num_observations)
    posterior_forecast_given_received_obs = np.zeros((agent.num_observations, agent.num_beliefs))
    for target_t in range(agent.num_beliefs):
        subterm1 = agent.within_trial_arrays['joint_prob_received_obs_state_t_from_targets'][agent.within_trial_params["time_ind"]] @ state_transition_matrix
        subterm2 = (agent.emission_matrix_targets[target_t,:] @ (subterm1).T)
        likelihood_obs_from_target_forecast_t_fromtargets = subterm2 @ target_transition_matrix[:,target_t]
        term2 = agent.within_trial_arrays['likelihood_received_obs_from_targets_t'][agent.within_trial_params["time_ind"], target_t]
        likelihood_obs_forecast_t_given_received_obs = agent.update_likelihood_given_observed_sequence(likelihood_obs_from_target_forecast_t_fromtargets, term2)

        updated_belief_after_action = agent.within_trial_arrays['posterior_t'][agent.within_trial_params["time_ind"]] @ target_transition_matrix

        # agent.within_trial_arrays['forecasted_obs'] = obs
        agent.within_trial_arrays['likelihood_obs_forecast_t_given_received_obs'] = likelihood_obs_forecast_t_given_received_obs
        # agent.debug_forecasting_each_obs_print_statements()
        
        numerator = (updated_belief_after_action[target_t] * likelihood_obs_forecast_t_given_received_obs)
        prob_obs_at_forecast_t_given_obs_at_cur_t[obs] = numerator.sum()
        posterior_forecast_given_received_obs[obs] = agent.bayes_update(numerator)
        forecasted_S[obs] = agent.compute_entropy_from_posterior_across_targets(posterior_forecast_given_received_obs[obs])

    return prob_obs_at_forecast_t_given_obs_at_cur_t, forecasted_S

In [16]:
agent.within_trial_arrays['posterior_t'][agent.within_trial_params["time_ind"]]

array([0., 0.])

In [13]:
agent.num_beliefs

2

In [10]:
forecast_t_candidate_step_states = -agent.state_step
forecast_t_candidate_step_targets = 'switch'
state_transition_matrix = (agent.get_deterministic_state_transition_matrix(forecast_t_candidate_step_states))
state_transition_matrix

array([[0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.]])

In [11]:
target_transition_matrix = get_deterministic_target_transition_matrix(forecast_t_candidate_step_targets)
target_transition_matrix

array([[0, 1],
       [1, 0]])

In [88]:
target_t_forecast_k = 1
agent.within_trial_arrays['joint_prob_received_obs_state_t_from_targets'][agent.within_trial_params["time_ind"]]

array([[0.25, 0.  , 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25]])

In [89]:
subterm1 = agent.within_trial_arrays['joint_prob_received_obs_state_t_from_targets'][agent.within_trial_params["time_ind"]] @ state_transition_matrix
subterm2 = (agent.emission_matrix_targets[target_t_forecast_k,:] @ (subterm1).T)
term1 = subterm2 @ target_transition_matrix[:,target_t_forecast_k]
term1

array([0.75, 0.  ])

In [90]:
term2 = agent.within_trial_arrays['likelihood_received_obs_from_targets_t'][agent.within_trial_params["time_ind"], target_t_forecast_k]
term2

np.float64(1.0)

In [91]:
agent.update_likelihood_given_observed_sequence(term1, term2)

array([0.75, 0.  ])

In [ ]:
def forecast_and_compute_expected_entropy_from_actions(agent):
    for i, forecast_t_candidate_step in enumerate(agent.candidate_steps):
        forecast_t_candidate_step_states = forecast_t_candidate_step[0]
        forecast_t_candidate_step_targets = forecast_t_candidate_step[1]
        state_transition_matrix = (agent.get_deterministic_state_transition_matrix(forecast_t_candidate_step_states))
        target_transition_matrix = (agent.get_deterministic_state_transition_matrix(forecast_t_candidate_step_targets))
        prob_obs_at_forecast_t_given_obs_at_cur_t, forecasted_S = compute_forecasted_entropy_from_forecasted_obs(agent, state_transition_matrix, target_transition_matrix)
        expected_forecast_t_S = (forecasted_S * prob_obs_at_forecast_t_given_obs_at_cur_t).sum()
        agent.print_debug_forecasted_entropy_statements(prob_obs_at_forecast_t_given_obs_at_cur_t, forecasted_S)
        agent.within_trial_arrays['expected_forecast_t_S_per_step'][agent.within_trial_params["time_ind"], i] = expected_forecast_t_S

In [17]:
across_trial_params = dict()
across_trial_params['trial_num'] = 0
across_trial_params['max_iter'] = 100
across_trial_params['num_trials'] = 10

across_trial_arrays = dict()
across_trial_arrays['posterior_across_trials'] = np.zeros((across_trial_params['num_trials'], across_trial_params['max_iter']+1, agent.num_beliefs))
across_trial_arrays['expected_forecast_t_across_trials'] = np.zeros((across_trial_params['num_trials'], across_trial_params['max_iter'], agent.candidate_steps.shape[0]))
across_trial_arrays['decision_type_across_trials'] = np.empty((across_trial_params['num_trials'], across_trial_params['max_iter']), dtype=np.dtypes.StringDType())
across_trial_arrays['angles_visited_across_trials'] = np.zeros((across_trial_params['num_trials'], across_trial_params['max_iter']))
across_trial_arrays['steps_taken_across_trials'] = np.zeros((across_trial_params['num_trials'], across_trial_params['max_iter']))
across_trial_arrays['entropy_across_trials'] = np.zeros((across_trial_params['num_trials'], across_trial_params['max_iter']))
across_trial_arrays['time_taken_per_trial'] = np.zeros(across_trial_params['num_trials'])
across_trial_arrays['codes_received_across_trials'] = np.zeros((across_trial_params['num_trials'], across_trial_params['max_iter']))

emission_matrix_true = EMISSION_MATRIX_TARGETS[0]

while across_trial_params['trial_num'] < across_trial_params['num_trials']:

    agent = HMM_target_detector(emission_matrix_targets=EMISSION_MATRIX_TARGETS)
    while agent.within_trial_params["time_ind"]<agent.within_trial_params['max_iter'] and agent.within_trial_params['stopping_num'] < 2:
        if np.isclose(agent.within_trial_arrays['posterior_t'][agent.within_trial_params["time_ind"]-1], 1, atol=1e-2).any() or agent.within_trial_params['stopping_num']>0:
            agent.within_trial_params['stopping_num']+=1

        agent.move_and_sample_from_object()

        agent.update_posterior_and_compute_current_entropy()

        agent.debug_within_trial_print_statements()

        forecast_and_compute_expected_entropy_from_actions(agent)

        agent.compute_expected_entropy_change_from_each_action()
        
        agent.determine_decision_from_forecasted_entropy_change()
        agent.within_trial_params["time_ind"]+=1
    
    summary_df = agent.get_summary_of_trial()
    summary_df.to_csv(f'{save_folder}/trial_{across_trial_params["trial_num"]}.csv')

    across_trial_arrays['posterior_across_trials'][across_trial_params['trial_num'],:agent.within_trial_params["time_ind"]+1,:] = np.vstack([agent.prior_target_prob, agent.within_trial_arrays['posterior_t'][:agent.within_trial_params["time_ind"]]])
    across_trial_arrays['entropy_across_trials'][across_trial_params['trial_num'], :agent.within_trial_params["time_ind"]] = agent.within_trial_arrays['current_entropyS_t'][:agent.within_trial_params["time_ind"]]
    across_trial_arrays['expected_forecast_t_across_trials'][across_trial_params['trial_num'], :agent.within_trial_params["time_ind"],:] = agent.within_trial_arrays['expected_forecast_t_S_per_step'][:agent.within_trial_params["time_ind"]]
    across_trial_arrays['steps_taken_across_trials'][across_trial_params['trial_num'],          :] = agent.within_trial_arrays['steps_taken']
    across_trial_arrays['angles_visited_across_trials'][across_trial_params['trial_num'],       :] = agent.within_trial_arrays['angles_visited']
    across_trial_arrays['codes_received_across_trials'][across_trial_params['trial_num'],       :] = agent.within_trial_arrays['code_received_t']
    across_trial_arrays['time_taken_per_trial'][across_trial_params['trial_num']] = agent.within_trial_params["time_ind"]
    across_trial_arrays['decision_type_across_trials'][across_trial_params['trial_num']] = agent.within_trial_arrays["decision_type_t"]
    across_trial_params['trial_num']+=1

Current angle: 0.0
Code received: 0.0
[0.375 0.5  ]
P(O1=[0.],S1|T1)=[0.25 0.   0.25 0.25], P(O1=[0.]|T1)=0.75, P(S1|O1=[0.],T1)=[0.33 0.   0.33 0.33], P(T|O1=[0.])=[0.42857143 0.57142857], H1=0.99
P(O1=[0.],S1|T2)=[0.25 0.25 0.25 0.25], P(O1=[0.]|T2)=1.0, P(S1|O1=[0.],T2)=[0.25 0.25 0.25 0.25], P(T|O1=[0.])=[0.42857143 0.57142857], H1=0.99


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 2 is different from 4)

In [9]:
test_df = pd.read_csv(f'{save_folder}/trial_0.csv', index_col=0)
test_df

,posterior_t_T1,posterior_t_T2,expected_forecast_t_S_per_step_CW,expected_forecast_t_S_per_step_STAY,expected_forecast_t_S_per_step_CCW,angles_visited,steps_taken,current_entropyS_t,code_received_t,decision_type_t,time_taken
0,0.428571,0.571429,0.787111,0.985228,0.787111,0.0,1.570796,0.985228,0.0,random,4.0
1,1.000000,0.000000,0.000000,0.000000,0.000000,90.0,0.000000,0.000000,1.0,end,4.0
2,1.000000,0.000000,0.000000,0.000000,0.000000,90.0,0.000000,0.000000,1.0,end,4.0
3,1.000000,0.000000,0.000000,0.000000,0.000000,90.0,0.000000,0.000000,1.0,end,4.0


In [ ]:
num_trials = len(list(save_folder.glob('*.csv')))
trial_selection_limit = min(10, num_trials)
trial_df_cache = {}

def get_summary_df(trial_idx):
    capped_idx = min(trial_idx, num_trials - 1)
    if capped_idx not in trial_df_cache:
        trial_df_cache[capped_idx] = pd.read_csv(f"{save_folder}/trial_{capped_idx}.csv", index_col=0)
    return trial_df_cache[capped_idx]

def update(time_ind, trial_num):
    summary_df_for_trial = get_summary_df(trial_num)
    fig = plt.figure(figsize=(12, 6))
    gs = gridspec.GridSpec(1, 2, width_ratios=[4, 4], wspace=0.3, figure=fig)
    gs_signals = gridspec.GridSpecFromSubplotSpec(
        2, 1, height_ratios=[2,1], subplot_spec=gs[0, 1], hspace=0.3)

    ax_grid = fig.add_subplot(gs[0, 0])
    ax_diffusion = fig.add_subplot(gs_signals[0])
    ax_entropy = fig.add_subplot(gs_signals[1])

    current_angle = (summary_df_for_trial['angles_visited'][time_ind]%360)
    code_received = summary_df_for_trial['code_received_t'][time_ind]
    initial_radians = np.radians(current_angle)

    setup_details = {'title': 'Target', 'grid_extent': 10, 
                        'inner_radius' : 1, 'outer_radius' : 5,
                        'agent_radians' : initial_radians}
    agent.plot_code_dependent_regions(ax_grid, EMISSION_MATRIX_TARGETS[0], setup_details)
    agent.plot_agent_in_env(ax_grid, setup_details, code_received)

    if time_ind>0:
        prev_angle = (summary_df_for_trial['angles_visited'][time_ind-1]%360)
        initial_radians = np.radians(prev_angle)
        ax_grid.scatter(1.2*setup_details['outer_radius']*np.cos(initial_radians), 1.2*setup_details['outer_radius']*np.sin(initial_radians),
                s=200, marker='.', facecolor='r', edgecolor='k', alpha=0.6, zorder=4)
        
    if time_ind>1:
        prev_angle = (summary_df_for_trial['angles_visited'][time_ind-2]%360)
        initial_radians = np.radians(prev_angle)
        ax_grid.scatter(1.4*setup_details['outer_radius']*np.cos(initial_radians), 1.4*setup_details['outer_radius']*np.sin(initial_radians),
                s=200, marker='.', facecolor='r', edgecolor='k', alpha=0.3, zorder=4)

    for i in range(agent.num_beliefs):
        if i == 0:
            class_name = 'target'
        else:
            class_name = 'non-target'
        ax_diffusion.plot(summary_df_for_trial[f'posterior_t_T{int(i+1)}'][:int(time_ind+1)], marker='.', label=f'Class {class_name}')
    ax_diffusion.axhline(y=1.0, linestyle='dashed', color='k')
    ax_diffusion.set_xlim(0, summary_df_for_trial['time_taken'][0]+1)
    ax_diffusion.set_ylim(0.0, 1.05)
    ax_diffusion.set_xticks(np.arange(summary_df_for_trial['time_taken'][0]+2).astype(int))
    ax_diffusion.set_ylabel('Class probability')
    ax_diffusion.set_xlabel('Time index (k)')

    x_vals = np.arange(max(0, time_ind-2), min(summary_df_for_trial.shape[0], time_ind+3)).astype(int)
    ax_entropy.plot(x_vals, summary_df_for_trial['expected_forecast_t_S_per_step_CW'][max(0, time_ind-2):min(summary_df_for_trial.shape[0], time_ind+3)], marker='.')
    ax_entropy.plot(x_vals, summary_df_for_trial['expected_forecast_t_S_per_step_STAY'][max(0, time_ind-2):min(summary_df_for_trial.shape[0], time_ind+3)], marker='.')
    ax_entropy.plot(x_vals, summary_df_for_trial['expected_forecast_t_S_per_step_CCW'][max(0, time_ind-2):min(summary_df_for_trial.shape[0], time_ind+3)], marker='.')
    ax_entropy.axvline(x=time_ind, linestyle='dashed', color='k', label=f"Decision type:{summary_df_for_trial['decision_type_t'][time_ind]}")
    xlabels = np.arange(time_ind-2, min(summary_df_for_trial.shape[0], time_ind+3)).astype(int)
    ax_entropy.set_xticks(xlabels)
    ax_entropy.set_ylabel('Forecasted entropy')
    ax_entropy.legend(loc='lower left')
    ax_entropy.set_xlabel('Time index (k)')


trial_selector = widgets.ToggleButtons(
    options=[(f'Trial {i}', i) for i in range(trial_selection_limit)],
    description='Trial',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

time_ind_slider = widgets.IntSlider(
    value=0, min=0, max=max(0, get_summary_df(0).shape[0]-1), step=1,
    description="Time (k)",
    continuous_update=True,
    style={'description_width': 'initial'},
    layout=widgets.Layout(width="400px")
)


def _sync_slider_with_trial():
    df = get_summary_df(trial_selector.value)
    new_max = max(0, df.shape[0]-1)
    time_ind_slider.max = new_max
    if time_ind_slider.value > new_max:
        time_ind_slider.value = new_max


def _on_trial_change(change):
    if change['name'] == 'value':
        _sync_slider_with_trial()


trial_selector.observe(_on_trial_change, names='value')
_sync_slider_with_trial()

controls = widgets.HBox([trial_selector, time_ind_slider])
interactive_plot = widgets.interactive_output(
    update, {'time_ind': time_ind_slider, 'trial_num': trial_selector})
display(widgets.VBox([controls, interactive_plot]))

